# Ingesting CSV data - Full Load

In [0]:
%run ../../config/project_config

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_BRONZE}.county_metrics (
        load_dt TIMESTAMP,
        source_file STRING
    )
    USING DELTA
""")

In [0]:
spark.sql(f"""
    COPY INTO {CATALOG_NAME}.{SCHEMA_BRONZE}.county_metrics
    FROM (
        SELECT 
            *,
            current_timestamp() AS load_dt,
            _metadata.file_path AS source_file
        FROM '{VOLUME_LANDING}/County_time_series_chunk_1.csv'
    )
    FILEFORMAT = CSV
    FORMAT_OPTIONS (
        'header' = 'true',
        'inferSchema' = 'true'
    )
    COPY_OPTIONS (
        'mergeSchema' = 'true',
        'force' = 'false'
    )
""")

In [0]:
spark.sql(f"""
    COMMENT ON COLUMN {CATALOG_NAME}.{SCHEMA_BRONZE}.county_metrics.load_dt 
    IS 'Governance: UTC Timestamp when this record was ingested into the Bronze layer.'
""")

spark.sql(f"""
    COMMENT ON COLUMN {CATALOG_NAME}.{SCHEMA_BRONZE}.county_metrics.source_file 
    IS 'Governance: The specific CSV file path in the Landing Volume that sourced this row.'
""")